# Exploring Information Theory in Experimental Design

Welcome to this tutorial on the basics of information theory and its practical application in optimizing experimental design! In this session, we'll delve into the fundamental concepts of information content, entropy, conditional entropy, and mutual information, all within the context of a synthetic working memory experiment.

#### Why should you even bother about Information Theory when designing an experiment?

Imagine you're designing an experiment to understand how human memory performs under different conditions. How do you choose the lengths of the sequences participants should remember? How do you quantify the information gained from each trial? This is where information theory steps in.

Information theory provides a powerful framework for measuring and quantifying uncertainty, information content, and the relationships between variables. In this tutorial, we'll guide you through the basics and show you how to apply these concepts in experimental design.

#### Optimizing the experimental design for a working memory experiment

Our context for learning will be a synthetic working memory experiment. 

In this experiment, participants (experimental units) are asked to remember a sequence of the length *L* which contains random numbers. 

We, as the researchers, have control about the sequence length *L*. This is the factor in our experiment.

The participants response will be categorized either being *Correct* (1) or *False* (0).

#### Tutorial Overview

In this tutorial you will learn to:
- Set up a synthetic working memory experiment (remember what you learned in the tutorial about synthetic experiments)
- Compute the information content and the entropy of the responses
- Compute the mutual information between our model parameters, the factors and the given responses
